In [17]:
import queue
import networkx as nx
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import messagebox
import heapq
import math
import numpy as np
import time
import threading

# Node Class

In [18]:
class PuzzleNode:
    def __init__(self, state, parent=None, move=None, g=0, h=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.g = g  # Depth
        self.h = h  # heuristic cost from current node to goal node
    
    def __eq__(self, other):
        return self.state == other.state
    
    def __lt__(self, other):
        # Compare nodes based on f = g + h
        return (self.g + self.h) < (other.g + other.h)
    
    def __hash__(self):
        return hash(str(self.state))
    
    def is_goal(self):
        return self.state == [[0,1,2],[3,4,5],[6,7,8]]
    

# Search Method Implementation

In [19]:
def get_neighbors(node):
    neighbors = []
    empty_row, empty_col = 0, 0
    node_state = node.state  # Extract the state from the PuzzleNode object
    for i in range(3):
        for j in range(3):
            if node_state[i][j] == 0:
                empty_row, empty_col = i, j
                break
    
    for dr, dc, move in [(1, 0, 'down'), (-1, 0, 'up'), (0, 1, 'right'), (0, -1, 'left')]:
        new_row, new_col = empty_row + dr, empty_col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_state = [row[:] for row in node_state]
            new_state[empty_row][empty_col] = new_state[new_row][new_col]
            new_state[new_row][new_col] = 0
            neighbors.append(new_state)
    
    return neighbors

## BFS

In [20]:
def bfs(initial_state):
    q = queue.Queue()
    q.put(PuzzleNode(initial_state, g=0))
    visited = set()
    num_expanded=0
    expanded=[]
    depth=0
    
    while not q.empty():
        current_node = q.get()
        if current_node.g > depth:
            depth= current_node.g
        visited.add(tuple(map(tuple, current_node.state)))  # Convert to tuple
        num_expanded +=1
        expanded.append(current_node)
        
        if current_node.is_goal():
            path = []
            cost= -1
            while current_node:
                path.append(current_node.state)
                cost+=1
                current_node = current_node.parent
            return list(reversed(path)), cost, num_expanded, expanded, depth
        
        for neighbor_state in get_neighbors(current_node):
            neighbor_node = PuzzleNode(neighbor_state, current_node, g= current_node.g+1)
            if tuple(map(tuple, neighbor_state)) not in visited:
                q.put(neighbor_node)
                visited.add(tuple(map(tuple, neighbor_state)))
    
    return None, None, None, None, None

## DFS

In [21]:
def dfs(initial_state):
    stack = [PuzzleNode(initial_state)]
    visited = set()
    num_expanded=0
    expanded= []
    depth=0
    while stack:
        current_node = stack.pop()
        if current_node.g > depth:
            depth= current_node.g
        visited.add(tuple(map(tuple, current_node.state)))  # Convert to tuple
        num_expanded+=1
        expanded.append(current_node)
        
        if current_node.is_goal():
            path = []
            cost = -1
            while current_node:
                path.append(current_node.state)
                cost += 1
                current_node = current_node.parent
            return list(reversed(path)), cost, num_expanded, expanded, depth
        
        for neighbor_state in get_neighbors(current_node):
            neighbor_node = PuzzleNode(neighbor_state, current_node, g= current_node.g+1)
            if tuple(map(tuple, neighbor_state)) not in visited:
                stack.append(neighbor_node)
                visited.add(tuple(map(tuple, neighbor_state)))
    
    return None, None, None, None, None


## A*

In [22]:
def euclidean_distance(state):
    # Calculate the Euclidean distance heuristic
    distance = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                goal_row = (state[i][j]) // 3
                goal_col = (state[i][j]) % 3
                distance += math.sqrt((i - goal_row)**2 + (j - goal_col)**2)
    return distance

def manhattan_distance(state):
    # Calculate the Manhattan distance heuristic
    distance = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                goal_row = (state[i][j]) // 3
                goal_col = (state[i][j]) % 3
                distance += abs(i - goal_row) + abs(j - goal_col)
    return distance


def astar_search(initial_state,f):
    frontier = []
    heapq.heappush(frontier, initial_state)
    visited = set()
    cost = -1
    num_expanded=0
    expanded=[]
    depth=0
    
    while frontier:
        current_node = heapq.heappop(frontier)
        if current_node.g > depth:
            depth= current_node.g
        visited.add(current_node)
        num_expanded+=1
        expanded.append(current_node)
        
        if tuple(map(tuple, current_node.state)) not in visited:
            visited.add(tuple(map(tuple, current_node.state)))
            for neighbor_state in get_neighbors(current_node):
                if f == 1:
                    neighbor = PuzzleNode(neighbor_state, current_node, g=current_node.g+1, h=manhattan_distance(neighbor_state))
                else:
                    neighbor = PuzzleNode(neighbor_state, current_node, g=current_node.g+1, h=euclidean_distance(neighbor_state))
                heapq.heappush(frontier, neighbor)

        if current_node.is_goal():
            path = []
            while current_node:
                path.append(current_node.state)
                current_node = current_node.parent
                cost+=1
            return list(reversed(path)),cost, num_expanded, expanded, depth
    
    return None,None, None, None, None


# Example usage

In [23]:
#goal=[[0,1,2],[3,4,5],[6,7,8]]

# npinitial_state = np.arange(9).reshape((3, 3))
# np.random.shuffle(npinitial_state)
# initial_state= npinitial_state.tolist()
# initial_state
    
#initial_state = [[3, 4, 5], [0, 1, 2], [6, 7, 8]]
initial_state = [[1,8,2],[0,4,3],[7,6,5]]

In [24]:
class PuzzleGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("8-Puzzle Solver")
        self.solution_states = []  # Store the solution states
        self.current_index = 0  # Current index of the solution_states list
        self.manual_nav_enabled = False  # Flag to enable manual navigation

        self.frame = tk.Frame(self.master)
        self.frame.pack()

        self.create_board()
        self.create_input()

    def create_board(self):
        self.board = [[tk.Button(self.frame, text="", width=5, height=2, command=self.tile_clicked(i, j)) for j in range(3)] for i in range(3)]
        for i in range(3):
            for j in range(3):
                self.board[i][j].config(bg="lightgray", fg="black", font=("Helvetica", 16, "bold"))
                self.board[i][j].grid(row=i, column=j)

    def create_input(self):
        input_frame = tk.Frame(self.master)
        input_frame.pack(pady=10)

        tk.Label(input_frame, text="Enter Initial State:").pack()

        self.initial_state_entry = tk.Text(input_frame, width=20, height=5)
        self.initial_state_entry.pack()

        solve_frame = tk.Frame(self.master)
        solve_frame.pack(pady=10)

        solve_bfs_button = tk.Button(solve_frame, text="Solve with BFS", command=self.solve_bfs, bg="blue", fg="white", font=("Helvetica", 12, "bold"))
        solve_bfs_button.grid(row=0, column=0, padx=5)

        solve_dfs_button = tk.Button(solve_frame, text="Solve with DFS", command=self.solve_dfs, bg="green", fg="white", font=("Helvetica", 12, "bold"))
        solve_dfs_button.grid(row=0, column=1, padx=5)

        solve_astarm_button = tk.Button(solve_frame, text="Solve with A* Manhattan", command=self.solve_astar_manhattan, bg="red", fg="white", font=("Helvetica", 12, "bold"))
        solve_astarm_button.grid(row=0, column=2, padx=5)

        solve_astare_button = tk.Button(solve_frame, text="Solve with A* Euclidean", command=self.solve_astar_euclidean, bg="orange", fg="white", font=("Helvetica", 12, "bold"))
        solve_astare_button.grid(row=0, column=3, padx=5)

        nav_frame = tk.Frame(self.master)
        nav_frame.pack(pady=5)

        backward_button = tk.Button(nav_frame, text="Backward", command=self.backward, bg="gray", fg="white", font=("Helvetica", 12, "bold"))
        backward_button.grid(row=0, column=0, padx=5)

        forward_button = tk.Button(nav_frame, text="Forward", command=self.forward, bg="gray", fg="white", font=("Helvetica", 12, "bold"))
        forward_button.grid(row=0, column=1, padx=5)

    def tile_clicked(self, row, col):
        def callback():
            number = self.initial_state[row][col]
            text = str(number) if number != 0 else ""  # Show empty tile as empty string
            self.board[row][col].config(text=text)
        return callback

    def get_initial_state(self):
        input_text = self.initial_state_entry.get("1.0", "end-1c").split("\n")
        initial_state = []
        for line in input_text:
            row = [int(x) for x in line.split()]
            initial_state.append(row)
        return initial_state

    def solve_bfs(self):
        self.initial_state = self.get_initial_state()
        bfs_queue = queue.Queue()
        t = threading.Thread(target=lambda: bfs_worker(self.initial_state, bfs_queue))
        t.start()
        self.master.after(1000, lambda: self.update_gui(bfs_queue))
    
    def solve_dfs(self):
        self.initial_state = self.get_initial_state()
        dfs_queue = queue.Queue()
        t = threading.Thread(target=lambda: dfs_worker(self.initial_state, dfs_queue))
        t.start()
        self.master.after(1000, lambda: self.update_gui(dfs_queue))

    def solve_astar_manhattan(self):
        self.initial_state = self.get_initial_state()
        astarm_queue = queue.Queue()
        t = threading.Thread(target=lambda: astar_worker(self.initial_state, astarm_queue, 1))
        t.start()
        self.master.after(1000, lambda: self.update_gui(astarm_queue))

    def solve_astar_euclidean(self):
        self.initial_state = self.get_initial_state()
        astare_queue = queue.Queue()
        t = threading.Thread(target=lambda: astar_worker(self.initial_state, astare_queue, 2))
        t.start()
        self.master.after(1000, lambda: self.update_gui(astare_queue))

    def update_gui(self, queue):
        if not queue.empty():
            result = queue.get()
            if result is not None:
                solution, cost, numExp, Ex, depth = result
                if solution:
                    self.solution_states = [state for state in solution]
                    self.current_index = 0
                    self.manual_nav_enabled = False  # Disable manual navigation during animation
                    self.update_board_with_delay()
                    messagebox.showinfo("Solution found", f"Cost={cost}\nNumber of nodes expanded= {numExp}\nSearch depth= {depth}")
                else:
                    messagebox.showinfo("No solution", "No solution found.")
        else:
            self.master.after(1000, lambda: self.update_gui(queue))

    def update_board_with_delay(self):
        if self.current_index < len(self.solution_states):
            current_state = self.solution_states[self.current_index]
            for i in range(3):
                for j in range(3):
                    number = current_state[i][j]
                    text = str(number) if number != 0 else ""  # Show empty tile as empty string
                    self.board[i][j].config(text=text)
            self.current_index += 1
            if self.current_index == len(self.solution_states):
                self.manual_nav_enabled = True  # Enable manual navigation after animation completes
            else:
                self.master.after(350, self.update_board_with_delay)  # 1ms delay between updates if dfs

    def backward(self):
        if self.manual_nav_enabled and self.current_index > 0:
            self.current_index -= 1
            self.update_board()

    def forward(self):
        if self.manual_nav_enabled and self.current_index < len(self.solution_states) - 1:
            self.current_index += 1
            self.update_board()

    def update_board(self):
        current_state = self.solution_states[self.current_index]
        for i in range(3):
            for j in range(3):
                number = current_state[i][j]
                text = str(number) if number != 0 else ""  # Show empty tile as empty string
                self.board[i][j].config(text=text)


def bfs_worker(initial_state, result_queue):
    start_time = time.time()
    result = bfs(initial_state)
    end_time = time.time()
    solution_bfs, cost_bfs, numExp_bfs, Ex_bfs, depth_bfs = result
    result_queue.put(result)
    if solution_bfs:
        print("BFS Solution found:")
        for state in solution_bfs:
            print(state[0])
            print(state[1])
            print(state[2])
            print("-----------")
        print(f"BFS Cost={cost_bfs} ")
        print(f"Number of nodes expanded= {numExp_bfs}")
        print(f"Search depth= {depth_bfs}")
        print(f"Time taken: {end_time - start_time} seconds")
    else:
        print("BFS No solution found.")

def dfs_worker(initial_state, result_queue):
    start_time = time.time()
    result = dfs(initial_state)
    end_time = time.time()
    solution_dfs, cost_dfs, numExp_dfs, Ex_dfs, depth_dfs = result
    result_queue.put(result)
    if solution_dfs:
        print("DFS Solution found:")
        for state in solution_dfs:
            print(state[0])
            print(state[1])
            print(state[2])
            print("-----------")
        print(f"DFS Cost={cost_dfs} ")
        print(f"Number of nodes expanded= {numExp_dfs}")
        print(f"Search depth= {depth_dfs}")
        print(f"Time taken: {end_time - start_time} seconds")
    else:
        print("DFS No solution found.")

def astar_worker(initial_state, result_queue, t):
    start_time = time.time()
    if t == 1:
        result = astar_search(PuzzleNode(initial_state, g=0, h=manhattan_distance(initial_state)), t)
    else:
        result = astar_search(PuzzleNode(initial_state, g=0, h=euclidean_distance(initial_state)), t)

    end_time = time.time()
    solution_astar, cost_astar, numExp_astar, Ex_astar, depth_astar = result
    result_queue.put(result)
    if t == 1:
        if solution_astar:
            print("A star manhattan Solution found:")
            for state in solution_astar:
                print(state[0])
                print(state[1])
                print(state[2])
                print("-----------")
            print(f"A star manhattan Cost={cost_astar} ")
            print(f"Number of nodes expanded= {numExp_astar}")
            print(f"Search depth= {depth_astar}")
            print(f"Time taken: {end_time - start_time} seconds")
        else:
            print("A star manhattan No solution found.")

    else:
        if solution_astar:
            print("A star euclidean Solution found:")
            for state in solution_astar:
                print(state[0])
                print(state[1])
                print(state[2])
                print("-----------")
            print(f"A star euclidean Cost={cost_astar} ")
            print(f"Number of nodes expanded= {numExp_astar}")
            print(f"Search depth= {depth_astar}")
            print(f"Time taken: {end_time - start_time} seconds")
        else:
            print("A star euclidean No solution found.")


def main():
    root = tk.Tk()
    app = PuzzleGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()


BFS No solution found.
BFS Solution found:
[1, 7, 3]
[2, 6, 8]
[0, 4, 5]
-----------
[1, 7, 3]
[2, 6, 8]
[4, 0, 5]
-----------
[1, 7, 3]
[2, 0, 8]
[4, 6, 5]
-----------
[1, 0, 3]
[2, 7, 8]
[4, 6, 5]
-----------
[1, 3, 0]
[2, 7, 8]
[4, 6, 5]
-----------
[1, 3, 8]
[2, 7, 0]
[4, 6, 5]
-----------
[1, 3, 8]
[2, 0, 7]
[4, 6, 5]
-----------
[1, 3, 8]
[0, 2, 7]
[4, 6, 5]
-----------
[0, 3, 8]
[1, 2, 7]
[4, 6, 5]
-----------
[3, 0, 8]
[1, 2, 7]
[4, 6, 5]
-----------
[3, 2, 8]
[1, 0, 7]
[4, 6, 5]
-----------
[3, 2, 8]
[0, 1, 7]
[4, 6, 5]
-----------
[3, 2, 8]
[4, 1, 7]
[0, 6, 5]
-----------
[3, 2, 8]
[4, 1, 7]
[6, 0, 5]
-----------
[3, 2, 8]
[4, 1, 7]
[6, 5, 0]
-----------
[3, 2, 8]
[4, 1, 0]
[6, 5, 7]
-----------
[3, 2, 0]
[4, 1, 8]
[6, 5, 7]
-----------
[3, 0, 2]
[4, 1, 8]
[6, 5, 7]
-----------
[3, 1, 2]
[4, 0, 8]
[6, 5, 7]
-----------
[3, 1, 2]
[4, 5, 8]
[6, 0, 7]
-----------
[3, 1, 2]
[4, 5, 8]
[6, 7, 0]
-----------
[3, 1, 2]
[4, 5, 0]
[6, 7, 8]
-----------
[3, 1, 2]
[4, 0, 5]
[6, 7, 8]
---